# LSGAN —— 最小二乘生成对抗网络

In [7]:
import argparse
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.utils as vu

from torch.autograd import Variable
from torchvision import transforms

## 前附 命令行参数
这个部分是通过命令行交互时可以查看的参数

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=32, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64, help="generator filter size")
parser.add_argument('--ndf', type=int, default=64, help="discriminator filter size")
parser.add_argument('--niter', type=int, default=100, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', default=True, action='store_true', help='enables cuda')
parser.add_argument('--outf', default='output/', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, help='manual seed')

opt = parser.parse_args(args=[])
# 打印参数
print(opt)

## 第一部分 Generator 和 Discriminator
这个部分是生成对抗网络最重要的两个组件。此处使用了卷积神经网络

In [ ]:
class Generator(nn.Module):
    def __init__(self, channel_number, generator_filter_number, latent_z):
        super(Generator, self).__init__()
        self.layer1 = nn.Sequential(nn.ConvTranspose2d(latent_z, generator_filter_number * 4, kernel_size=4),
                                    nn.BatchNorm2d(generator_filter_number * 4),
                                    nn.ReLU())
        # 4 x 4 子网
        self.layer2 = nn.Sequential(nn.ConvTranspose2d(generator_filter_number * 4, generator_filter_number * 2, kernel_size=4, stride=2, padding=1),
                                    nn.BatchNorm2d(generator_filter_number * 2),
                                    nn.ReLU())
        # 8 x 8 子网
        self.layer3 = nn.Sequential(nn.ConvTranspose2d(generator_filter_number * 2, generator_filter_number, kernel_size=4, stride=2, padding=1),
                                    nn.BatchNorm2d(generator_filter_number),
                                    nn.ReLU())

        # 16 x 16 子网
        self.layer4 = nn.Sequential(nn.ConvTranspose2d(generator_filter_number, channel_number, kernel_size=4, stride=2, padding=1),
                                    nn.Tanh())

    # 构建神经网络
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        return out

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channel_number, discriminator_filter_number):
        super(Discriminator, self).__init__()
        # 32 x 32 子网
        self.layer1 = nn.Sequential(nn.Conv2d(channel_number, discriminator_filter_number, kernel_size=4, stride=2, padding=1),
                                    nn.BatchNorm2d(discriminator_filter_number),
                                    nn.LeakyReLU(0.2, inplace=True))
        # 16 x 16 子网
        self.layer2 = nn.Sequential(nn.Conv2d(discriminator_filter_number, discriminator_filter_number * 2, kernel_size=4, stride=2, padding=1),
                                    nn.BatchNorm2d(discriminator_filter_number * 2),
                                    nn.LeakyReLU(0.2, inplace=True))
        # 8 x 8 子网
        self.layer3 = nn.Sequential(nn.Conv2d(discriminator_filter_number * 2, discriminator_filter_number * 4, kernel_size=4, stride=2, padding=1),
                                    nn.BatchNorm2d(discriminator_filter_number * 4),
                                    nn.LeakyReLU(0.2, inplace=True))
        # 4 x 4 子网
        self.layer4 = nn.Sequential(nn.Conv2d(discriminator_filter_number * 4, 1, kernel_size=4, stride=1, padding=0))

    # 构建神经网络
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

## 第二部分 初始化
这部分是初始化许多参数，为训练做准备

In [ ]:
# 创建文件夹，按照参数，默认为output文件夹
# 如果没有output文件夹，那么久创建一个文件夹
try:
    os.makedirs(opt.outf)
except OSError:
    pass

# 设置随机种子，这个地方也可以手动设置
if opt.manualSeed is None:
    opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
if opt.cuda and torch.cuda.is_available():
    torch.cuda.manual_seed_all(opt.manualSeed)

# GPU加速
cudnn.benchmark = True

# 正则化图形变换
data_transform = transforms.Compose([
    transforms.Resize(opt.imageSize),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 通过文件获取数据集
data_dir = "./dataset/animation/"
data_set = dset.ImageFolder(data_dir, data_transform)
# 这个data.DataLoader不要修改，runtime可知！
loader = torch.utils.data.DataLoader(data_set, batch_size=opt.batchSize, shuffle=True, num_workers=8)

# 构建训练模型
# generator中filter的数量
input_generator_filter_number = opt.ngf
# discriminator中filter的数量
input_discriminator_filter_number = opt.ndf
# 输入图像的channel
input_channel_number = 3

generator = Generator(input_channel_number, input_generator_filter_number, opt.nz)
discriminator = Discriminator(input_channel_number, input_discriminator_filter_number)

# 损失函数与优化设置
criterion = nn.BCELoss()  # BCE => binary cross entropy
optimizerG = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerD = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))

# 全局变量
noise = torch.FloatTensor(opt.batchSize, opt.nz, 1, 1)
real = torch.FloatTensor(opt.batchSize, input_channel_number, opt.imageSize, opt.imageSize)
label = torch.FloatTensor(opt.batchSize)
real_label = 1
fake_label = 0

noise = Variable(noise)
real = Variable(real)
label = Variable(label)

# 如果GPU是能够使用的，那么使用GPU去训练
if opt.cuda and torch.cuda.is_available():
    print("cuda is available")
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    noise = noise.cuda()
    real = real.cuda()
    label = label.cuda()

# 使用list记录一下loss值
generator_loss = []
discriminator_loss = []

# 计时器
start_time = datetime.datetime.now()

## 第三部分 训练
训练的epoch是一个超参数，这里训练的过程参考最经典的GAN就可以了

In [ ]:
for epoch in range(1, opt.niter + 1):
    for i, (images, _) in enumerate(loader):
        # fDx
        discriminator.zero_grad()
        # 使用真实数据训练，这里考虑到可能最后图片一个batch的数量会小于 opt.batchSize
        with torch.no_grad():
            real.resize_(images.size()).copy_(images)
            label.resize_(images.size(0)).fill_(real_label)

        output = discriminator(real)

        errD_real = 0.5 * torch.mean((output - label) ** 2)
        errD_real.backward()

        # 使用虚假的数据训练
        label.data.fill_(fake_label)
        with torch.no_grad():
            noise.resize_(images.size(0), opt.nz, 1, 1)
        noise.data.normal_(0, 1)

        fake = generator(noise)
        # 在这里解离梯度，这样generator的梯度不会更新
        output = discriminator(fake.detach())

        errD_fake = 0.5 * (torch.mean((output - label)) ** 2)
        errD_fake.backward()

        errD = errD_fake + errD_real
        optimizerD.step()

        # fGx
        generator.zero_grad()
        label.data.fill_(real_label)
        output = discriminator(fake)

        errG = 0.5 * (torch.mean(output - label) ** 2)
        errG.backward()
        optimizerG.step()

        # 输出日志信息
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f ' % (epoch, opt.niter, i, len(loader), errD, errG))

        generator_loss.append(errG)
        discriminator_loss.append(errD)

        # 可视化（每300次做一次可视化）
        if i % 300 == 0:
            vu.save_image(fake.data, '%s/fake_samples_epoch_%03d_iter%03d.png' % (opt.outf, epoch, i), normalize=True)

In [ ]:
 # 计时结束
end_time = datetime.datetime.now()

## 第四部分 输出结果

In [ ]:
print("spent {} minutes".format((end_time - start_time).seconds / 60))
# 保存generator和discriminator的状态，便于读取
torch.save(generator.state_dict(), '%s/netG.pth' % opt.outf)
torch.save(discriminator.state_dict(), '%s/netD.pth' % opt.outf)

# 绘制学习曲线
generator_loss_numpy = []
discriminator_loss_numpy = []

# 因为CUDA tensor不能直接转换为numpy数组，所以需要先将其转换成cpu float-tensor随后再转到numpy格式
for i in generator_loss:
    generator_loss_numpy.append(i.data.cpu().numpy())

for i in discriminator_loss:
    discriminator_loss_numpy.append(i.data.cpu().numpy())

plt.plot(generator_loss_numpy, label="Generator")
plt.plot(discriminator_loss_numpy, label="Discriminator")
plt.xlabel("iterations")
plt.ylabel("loss")
plt.legend(loc="upper right")
plt.savefig(opt.outf + "loss")
plt.close()

# 将损失函数的信息存储为txt
np.savetxt(opt.outf + "g_loss.txt", generator_loss_numpy)
np.savetxt(opt.outf + "d_loss.txt", discriminator_loss_numpy)

# 平滑loss函数曲线
N = 100
g_loss_smooth = np.convolve(generator_loss_numpy, np.ones((N,)) / N, mode='valid')
d_loss_smooth = np.convolve(discriminator_loss_numpy, np.ones((N,)) / N, mode='valid')

np.savetxt(opt.outf + "g_loss_smooth.txt", g_loss_smooth)
np.savetxt(opt.outf + "d_loss_smooth.txt", d_loss_smooth)

plt.plot(g_loss_smooth, label="Generator")
plt.plot(d_loss_smooth, label="Discriminator")
plt.xlabel("iterations")
plt.ylabel("loss")
plt.legend(loc="upper right")
plt.savefig(opt.outf + "loss(smooth)")
plt.close()